In [5]:
%matplotlib inline
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

import matplotlib.image as mpimg
from skimage.io import imread, imshow
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
from scipy import ndimage
import seaborn as sns
#import Tensorflow namespaces

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

import pickle

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
def evaluate_loss(history, metric):
    # Loss 
    metric_loss = "val_" + metric
    plt.plot(history.history[metric],'r')
    plt.plot(history.history[metric_loss],'b')

    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [3]:
def load_images(path):

    images = []
    for image_file in os.listdir(path):

        image = io.imread(os.path.join(path, image_file))
        if image is not None:
            images.append(image)
            

    return images

box = load_images('./dataset/box')
lines = load_images('./dataset/lines')

In [4]:
# Normalization of values
Xs = box + lines
Xs = np.array(Xs)

Xs = Xs.astype('float32')
Xs /= 255


In [5]:
# Generate labels 1 for vehicles and 0 for non-vehicles
class_labels = np.array([1] * len(box) + [0] * len(lines))
print(class_labels)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
# Devide the Xs into training and test sets
X_train, X_test, y_train, y_test = train_test_split(Xs, class_labels, test_size=0.2, random_state=20)

y_train = to_categorical(y_train)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(160, 36, 36, 4)
(40, 36, 36, 4)
(160, 2)
(40,)


In [7]:
METRICS = [ 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [8]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)

# Neural network parameters
#-----------------------------------------------
#-----------------------------------------------
batch_size = 64 #
epochs = 50 # 

kernel_size=(3,3)
#-----------------------------------------------
#-----------------------------------------------
num_classes = 2
input_shape = (36, 36, 4)

# Model
model = Sequential()
model.add(Conv2D(16, kernel_size=kernel_size, input_shape=input_shape)) 
model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=kernel_size)) 

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten()) 
model.add(Dropout(0.4)) 
model.add(Dense(20, activation='relu')) 

model.add(Dropout(0.4))

model.add(Dense(num_classes, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, min_delta=0.001, patience=10, restore_best_weights=True)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=METRICS)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 34, 34, 16)        592       
                                                                 
 batch_normalization (BatchN  (None, 34, 34, 16)       64        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 34, 34, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 17, 17, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 16)        2320      
                                                                 
 batch_normalization_1 (Batc  (None, 15, 15, 16)       6

In [9]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/50


In [ ]:
evaluate_loss(history, "loss")
evaluate_loss(history, "accuracy")
evaluate_loss(history, "precision")
evaluate_loss(history, "recall")
evaluate_loss(history, "auc")